In [30]:
from fastapi import FastAPI
import pickle
import numpy as np
from pydantic import BaseModel

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
import pickle

In [32]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

X_train = train_df.drop('Cover_Type', axis=1)
y_train = train_df['Cover_Type']

X_test = test_df

In [33]:
lgb_model = LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)

mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)

ensemble_model = VotingClassifier(estimators=[
    ('lgb', lgb_model),
    ('mlp', mlp_model)
], voting='hard')
ensemble_model.fit(X_train, y_train)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000460 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2426
[LightGBM] [Info] Number of data points in the train set: 15120, number of used features: 46
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000130 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2426
[LightGBM] [Info] Number of data points in the train set: 15120, nu

VotingClassifier(estimators=[('lgb', LGBMClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [34]:
lgb_predictions = lgb_model.predict(X_test)
mlp_predictions = mlp_model.predict(X_test)
ensemble_predictions = ensemble_model.predict(X_test)

# You can evaluate the models using any appropriate metrics
# For this example, let's just print the predicted classes
print("LightGBM Predictions:", lgb_predictions)
print("MLP Predictions:", mlp_predictions)
print("Ensemble Predictions:", ensemble_predictions)

with open('lgb_model.pkl', 'wb') as f:
    pickle.dump(lgb_model, f)

with open('mlp_model.pkl', 'wb') as f:
    pickle.dump(mlp_model, f)

with open('ensemble_model.pkl', 'wb') as f:
    pickle.dump(ensemble_model, f)

LightGBM Predictions: [1 1 1 ... 3 3 3]
MLP Predictions: [5 5 2 ... 3 3 3]
Ensemble Predictions: [1 1 1 ... 3 3 3]


In [35]:
with open('lgb_model.pkl', 'rb') as f:
    model1 = pickle.load(f)

with open('mlp_model.pkl', 'rb') as f:
    model2 = pickle.load(f)

with open('ensemble_model.pkl', 'rb') as f:
    ensemble_model = pickle.load(f)

In [36]:
app = FastAPI()

class InputFeatures(BaseModel):
    elevation: float
    aspect: float
    slope: float
    horizontal_distance_to_hydrology: float
    vertical_distance_to_hydrology: float
    horizontal_distance_to_roadways: float
    hillshade_9am: float
    hillshade_noon: float
    hillshade_3pm: float
    horizontal_distance_to_fire_points: float
    wilderness_area: list[int]
    soil_type: list[int]

class PredictionResult(BaseModel):
    predicted_class: int

@app.get("/")
def read_root():
    return {"message": "Welcome to the Forest Cover Type Prediction API"}

@app.get("/version")
def get_version():
    return {"version": "1.0"}

@app.post("/predict1", response_model=PredictionResult)
def predict_top_classifier(data: InputFeatures):
    features = np.array([list(data.dict().values())[:-2]]).astype(np.float32)
    prediction = model1.predict(features)
    return {"predicted_class": int(prediction[0])}

@app.post("/predict2", response_model=PredictionResult)
def predict_second_best_classifier(data: InputFeatures):
    features = np.array([list(data.dict().values())[:-2]]).astype(np.float32)
    prediction = model2.predict(features)
    return {"predicted_class": int(prediction[0])}

@app.post("/predict3", response_model=PredictionResult)
def predict_ensemble_classifier(data: InputFeatures):
    features = np.array([list(data.dict().values())[:-2]]).astype(np.float32)
    prediction = ensemble_model.predict(features)
    return {"predicted_class": int(prediction[0])}

In [ ]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [18]:
import requests

data = {
    "elevation": 3000.0,
    "aspect": 180.0,
    "slope": 15.0,
    "horizontal_distance_to_hydrology": 120.0,
    "vertical_distance_to_hydrology": -10.0,
    "horizontal_distance_to_roadways": 200.0,
    "hillshade_9am": 220.0,
    "hillshade_noon": 250.0,
    "hillshade_3pm": 150.0,
    "horizontal_distance_to_fire_points": 500.0,
    "wilderness_area": [0, 1, 0, 0],  # Example wilderness area
    "soil_type": [1 if i == 2 else 0 for i in range(40)]  # Example soil type
}

response1 = requests.post("http://localhost:8000/predict1", json=data)
response2 = requests.post("http://localhost:8000/predict2", json=data)
response3 = requests.post("http://localhost:8000/predict3", json=data)

print("Predict 1:", response1.json())
print("Predict 2:", response2.json())
print("Predict 3:", response3.json())


ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /predict1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022B14263550>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))